In [ ]:
!pip install pydeck turfpy

In [ ]:
import numpy as np
import pandas as pd

df = pd.read_csv('../input/uber-pickups-in-new-york-city/uber-raw-data-apr14.csv')
df.head()

In [ ]:
df.describe()

In [ ]:
from kaggle_secrets import UserSecretsClient
import os
user_secrets = UserSecretsClient()
os.environ['MAPBOX_API_KEY']=user_secrets.get_secret("mapbox")

In [ ]:
import pydeck as pdk

midpoint = (np.average(df["Lat"]), np.average(df["Lon"]))
zoom = 11

initial_view_state={
    "latitude": midpoint[0],
    "longitude": midpoint[1],
    "zoom": zoom
}

heatmap_layer = pdk.Layer(
    "HeatmapLayer",
    data=df[['Lon', 'Lat']],
    opacity=0.9,
    get_position=['Lon', 'Lat'],
    pickable=True,
)

map = pdk.Deck(
    initial_view_state=initial_view_state,
    layers=[heatmap_layer],
    map_provider="mapbox",
    # map_style=pdk.map_styles.SATELLITE
)

map.to_html('map.html')

In [ ]:
from turfpy.transformation import circle
from geojson import Point, Feature, FeatureCollection

center_point = Point((midpoint[1], midpoint[0]))

circles = [
    circle(
        center=Feature(geometry=center_point),
        radius=radius,
        steps=100,
        units='km'
    )
    for radius in range(1, 21)
]
geo_json_data=FeatureCollection(circles)

In [ ]:
geo_json_layer = pdk.Layer(
    "GeoJsonLayer",
    geo_json_data,
    filled=False,
    stroked=True,
    get_line_color=[150, 150, 150],
    get_line_width=50,
)

map = pdk.Deck(
    initial_view_state=initial_view_state,
    layers=[
        heatmap_layer,
        geo_json_layer
    ],
    map_provider="mapbox",
    # map_style=pdk.map_styles.SATELLITE
)

map.to_html('map.html')